In [24]:
# OUR TASK. Import request & beutiful soup
import requests
import bs4
def create_soup(url,tag_name):
    page = requests.get(url)
    soup = bs4.BeautifulSoup(page.text,"lxml")
    result = soup.select(tag_name)
    return result

def get_row_label(current_country, current_province):
    if current_province == '':
        result = current_country
    else:
        result = current_country+':-'+current_province
    return result

# SEARCH ITEMS
def find_exact(dictionary, search_string):
    result = dictionary.get(search_string)
    return result

def find_wildcard(dictionary,search_string):
    output_dictionary={}
    for key,val in dictionary.items():
        if search_string in key:
            output_dictionary[key] = val
    return output_dictionary
            
def get_covstats(url,tag):
    data = create_soup(url,tag) 
    #Create Resulting List
    master_dictionary = {}
    country_lookup = []
    for_dataframe={}
    simple_list_output=[]
    #Create Header Row
    headers = data[0].getText()
    headers_list = headers.split('\n')
    headers_list_cleaned = headers_list[2:10]
    # this data[1:5] is just 3 rows for testing purposes
    for item in data[1:4]:
        item_text = item.getText()
        item_list = item_text.split('\n')
        #item_list[3-5] is limiting the columns for testing purposes
        item_list_cleaned = item_list[2:10]
    
        dictionary_row = dict(zip(headers_list_cleaned, item_list_cleaned))

        # put it into summary list
        current_province =  dictionary_row["Province/State"]
        current_country = dictionary_row["Country/Region"] 
        row_label = get_row_label(current_country, current_province)
        item_number = item.key
        #print(item_number)
        #1. build a SET of countries (FOR LOOKUP)
        country_lookup.append(row_label) 
        #2 Build object of labeled lists (adding row_label)
        master_dictionary.update({row_label:dictionary_row})
        for_dataframe.update({row_label:item_list_cleaned})
        #3 Build just a list
        simple_list_output.append(item_list_cleaned)
        
    return [master_dictionary, country_lookup, for_dataframe,simple_list_output,headers_list_cleaned]

In [25]:
url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
tag = '.js-file-line'

result = get_covstats(url,tag)
#print(result[3])


#WILDCARD SEARCH
#wildcard_match = find_wildcard(result[0],'Ang')
#print(wildcard_match)

# REQUIRED FORMAT
#data = {'City':['SF','LA','NYC'],
#        'Population':[837000,3880000,8400000]}

# Should be
#        {
#           country:[afghanistan,UK],
#           province:['',Isle of Mann]
#
#}

#PLAN
#1 TAKE result as input
#2. loop around list
#3 map(list, zip(*l))
#--> [[1, 4, 7], [2, 5, 8], [3, 6, 9]]


In [35]:
# TAKE THE OUTPUT AND RECOFIGURE
# get headers
headers = result[4]
#print(headers)
# list(map(list, zip(*l)))
transposed_result = list(map(list, zip(*result[3])))
#print(transposed_result)
#zip the Headers to the data

data_frame={}
for n in range(3):   
    data_frame.update({headers[n]:transposed_result[n]})
print(data_frame)
#1. Turn the above into a function 
#2.Next step make a dataframe
          

{'Province/State': ['', '', ''], 'Country/Region': ['Afghanistan', 'Albania', 'Algeria'], 'Lat': ['33.93911', '41.1533', '28.0339']}


In [6]:
# Creating or Dataframe
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
#EXISTING
# {
#UK-IsleOfMann:{},
#AFGANISTAN:{},
#FRANCE:{}
#}

#INTERIM NEW FORMAT
#{
#    [UK,isl of....],
#    [Afganistan...],
#    [France ...]
#}

#FINAL NEW FORMAT
#{
#    country:[uk,Afganistan,france],
#    province:[Isle of Mann, "",""],
#    '01/01/20120':[32,3,222]
#}

#Lets create the INTERIM NEW FORMAT FROM 

